# Motor de Stress Testing en Python
## Escenarios de Estres y Cambios de Regimen de Mercado

**Practica B2-2 - Modulo de Gestion de Riesgos**

| | |
|---|---|
| **Autor** | Raul Rodriguez |
| **Fecha** | Febrero 2026 |
| **Periodo de datos** | 01-01-2006 a la fecha |

In [1]:
# Descomentar en Google Colab:
# !pip install hmmlearn yfinance copulas -q

---
## 1. Configuracion

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
from scipy.stats import skew, kurtosis, kendalltau
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

COLOR_CALMA = '#2E86AB'
COLOR_CRISIS = '#E94F37'

SEED = 42
np.random.seed(SEED)

START_DATE = '2006-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
N_SIMULATIONS = 10_000
HORIZON_DAYS = 126

print(f"Periodo: {START_DATE} a {END_DATE}")
print(f"Simulaciones: {N_SIMULATIONS:,} | Horizonte: {HORIZON_DAYS} dias")

Periodo: 2006-01-01 a 2026-02-06
Simulaciones: 10,000 | Horizonte: 126 dias


In [3]:
PORTFOLIO_TICKERS = {
    'AAPL': 'Apple', 'AMZN': 'Amazon', 'BAC': 'Bank of America',
    'BRK-B': 'Berkshire', 'CVX': 'Chevron', 'ENPH': 'Enphase',
    'GLD': 'Gold ETF', 'GME': 'GameStop', 'GOOGL': 'Alphabet',
    'JNJ': 'J&J', 'JPM': 'JPMorgan', 'MSFT': 'Microsoft',
    'NVDA': 'NVIDIA', 'PG': 'P&G', 'XOM': 'ExxonMobil',
    'IEF': 'Treasury 10Y', 'SHY': 'Treasury 2Y', 'HYG': 'High Yield'
}
MARKET_TICKER = '^GSPC'
TICKERS = list(PORTFOLIO_TICKERS.keys())
print(f"Universo: {len(TICKERS)} activos")

Universo: 18 activos


---
## 2. Datos

In [ ]:
all_tickers = TICKERS + [MARKET_TICKER]
print(f"Descargando {len(all_tickers)} activos...")
prices = yf.download(all_tickers, start=START_DATE, end=END_DATE, progress=True, auto_adjust=True)['Close']
if isinstance(prices.columns, pd.MultiIndex):
    prices.columns = prices.columns.droplevel(0)
print(f"Datos: {len(prices)} observaciones")

Descargando 19 activos...


In [ ]:
returns = np.log(prices / prices.shift(1)).dropna()
print(f"Retornos: {len(returns)} obs")

summary = pd.DataFrame({
    'Media anual (%)': returns.mean() * 252 * 100,
    'Vol anual (%)': returns.std() * np.sqrt(252) * 100
}).round(2)
display(summary)

---
## 3. Deteccion de regimenes (HMM)

In [ ]:
market_returns = returns[MARKET_TICKER].values.reshape(-1, 1)

hmm_model = GaussianHMM(n_components=2, covariance_type='full', n_iter=100, random_state=SEED)
hmm_model.fit(market_returns)
states = hmm_model.predict(market_returns)

vol_0 = market_returns[states == 0].std()
vol_1 = market_returns[states == 1].std()
if vol_0 > vol_1:
    states = 1 - states
    hmm_model.means_ = hmm_model.means_[::-1]
    hmm_model.covars_ = hmm_model.covars_[::-1]
    hmm_model.transmat_ = hmm_model.transmat_[::-1, ::-1]

returns['State'] = states

vol_calma = market_returns[states == 0].std()
vol_crisis = market_returns[states == 1].std()
print(f"Calma: vol={vol_calma:.4f} | Crisis: vol={vol_crisis:.4f} | Ratio: {vol_crisis/vol_calma:.2f}x")

In [ ]:
trans_df = pd.DataFrame(hmm_model.transmat_, index=['Calma', 'Crisis'], columns=['->Calma', '->Crisis'])
display(trans_df.round(4))

dur_calma = 1 / hmm_model.transmat_[0, 1]
dur_crisis = 1 / hmm_model.transmat_[1, 0]
print(f"Duracion calma: {dur_calma:.0f} dias | crisis: {dur_crisis:.0f} dias")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 8), gridspec_kw={'height_ratios': [3, 1]})
price_data = prices[MARKET_TICKER].loc[returns.index]

ax1 = axes[0]
ax1.plot(price_data.index, price_data.values, color='black', linewidth=0.8, alpha=0.7)
for i in range(len(states)):
    color = COLOR_CALMA if states[i] == 0 else COLOR_CRISIS
    alpha = 0.15 if states[i] == 0 else 0.25
    ax1.axvspan(price_data.index[i], price_data.index[min(i+1, len(states)-1)], color=color, alpha=alpha)
ax1.set_title('S&P 500 con Regimenes Detectados', fontsize=14, fontweight='bold')
ax1.legend(handles=[Patch(facecolor=COLOR_CALMA, alpha=0.3, label='Calma'), Patch(facecolor=COLOR_CRISIS, alpha=0.4, label='Crisis')])

axes[1].fill_between(price_data.index, states, color=COLOR_CRISIS, alpha=0.6, step='post')
axes[1].set_yticks([0, 1]); axes[1].set_yticklabels(['Calma', 'Crisis'])
plt.tight_layout()
plt.show()

print(f"Calma: {(states==0).sum()} dias ({(states==0).mean():.1%}) | Crisis: {(states==1).sum()} dias")

---
## 4. Analisis marginal por estado

In [ ]:
returns_data = returns.drop(columns=['State'])
returns_calma = returns_data[states == 0]
returns_crisis = returns_data[states == 1]

vol_calma_df = returns_calma.std() * np.sqrt(252) * 100
vol_crisis_df = returns_crisis.std() * np.sqrt(252) * 100

vol_comparison = pd.DataFrame({
    'Vol Calma (%)': vol_calma_df,
    'Vol Crisis (%)': vol_crisis_df,
    'Ratio': vol_crisis_df / vol_calma_df
}).sort_values('Ratio', ascending=False).round(2)

display(vol_comparison)
print(f"Promedio: volatilidad aumenta {vol_comparison['Ratio'].mean():.1f}x en crisis")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
assets = ['GLD', 'HYG', MARKET_TICKER, 'NVDA', 'BAC', 'IEF']
for idx, asset in enumerate(assets):
    ax = axes[idx // 3, idx % 3]
    ax.hist(returns_calma[asset]*100, bins=50, alpha=0.6, color=COLOR_CALMA, label='Calma', density=True)
    ax.hist(returns_crisis[asset]*100, bins=50, alpha=0.6, color=COLOR_CRISIS, label='Crisis', density=True)
    ax.set_title(asset, fontweight='bold')
    if idx == 0: ax.legend()
plt.suptitle('Distribucion de Retornos por Estado', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print("ORO (GLD) como refugio:")
print(f"  Corr vs S&P: Calma={returns_calma['GLD'].corr(returns_calma[MARKET_TICKER]):.3f}, Crisis={returns_crisis['GLD'].corr(returns_crisis[MARKET_TICKER]):.3f}")
print(f"\nHIGH YIELD (HYG):")
print(f"  Corr vs S&P: Calma={returns_calma['HYG'].corr(returns_calma[MARKET_TICKER]):.3f}, Crisis={returns_crisis['HYG'].corr(returns_crisis[MARKET_TICKER]):.3f}")

---
## 5. Estructura de dependencia (Copulas)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 7))
corr_calma = returns_calma[TICKERS].corr()
corr_crisis = returns_crisis[TICKERS].corr()

sns.heatmap(corr_calma, ax=axes[0], cmap='RdBu_r', center=0, vmin=-1, vmax=1, square=True)
axes[0].set_title('Correlaciones en CALMA', fontsize=14, fontweight='bold')

sns.heatmap(corr_crisis, ax=axes[1], cmap='RdBu_r', center=0, vmin=-1, vmax=1, square=True)
axes[1].set_title('Correlaciones en CRISIS', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

mask = np.triu(np.ones_like(corr_calma, dtype=bool), k=1)
print(f"Corr media: Calma={corr_calma.where(mask).stack().mean():.3f}, Crisis={corr_crisis.where(mask).stack().mean():.3f}")

In [ ]:
# Seleccionar activos para simulacion
selected = [MARKET_TICKER, 'NVDA', 'BAC', 'GLD', 'HYG', 'IEF']
data_calma = returns_calma[selected].dropna()
data_crisis = returns_crisis[selected].dropna()
print(f"Activos seleccionados: {len(selected)} | Obs calma: {len(data_calma)}, crisis: {len(data_crisis)}")

---
## 6. Motor Monte Carlo

### Nota metodologica: Muestreo bootstrap vs copulas

Para generar retornos simulados condicionados al estado del mercado, se dispone de dos enfoques principales:

**Muestreo desde copula ajustada** (copula.sample()): Se ajusta una copula (gaussiana, t-Student, etc.) a los datos historicos de cada estado y se generan muestras sinteticas a partir de ella. Este metodo captura la estructura de dependencia parametricamente, pero es computacionalmente costoso cuando se requieren millones de muestras.

**Muestreo bootstrap**: Se seleccionan aleatoriamente observaciones de los datos historicos con reemplazo. Este metodo no parametrico preserva todas las caracteristicas empiricas de los datos (distribucion marginal, dependencia, colas) sin necesidad de ajustar un modelo explicito.

Ambos enfoques son validos para simulacion Monte Carlo. En este trabajo se utiliza bootstrap por su eficiencia computacional, lo que permite ejecutar 10.000 trayectorias en tiempo razonable. La estructura de dependencia queda implicitamente capturada al muestrear vectores completos de retornos multiactivo de cada estado.


In [ ]:
def simulate_portfolio(n_sim, horizon, transmat, data_calma, data_crisis, initial_state=0):
    """
    Simula trayectorias de cartera con cambio de regimen.
    Usa muestreo bootstrap de datos historicos (eficiente).
    """
    n_assets = data_calma.shape[1]
    weights = np.ones(n_assets) / n_assets
    
    # Pre-generar indices aleatorios
    idx_calma = np.random.randint(0, len(data_calma), size=(n_sim, horizon))
    idx_crisis = np.random.randint(0, len(data_crisis), size=(n_sim, horizon))
    rand_trans = np.random.random((n_sim, horizon))
    
    portfolio_values = np.ones((n_sim, horizon + 1))
    
    for sim in range(n_sim):
        state = initial_state
        cumret = 0.0
        for t in range(horizon):
            # Transicion de estado
            if rand_trans[sim, t] < transmat[state, 1-state]:
                state = 1 - state
            
            # Muestrear retornos historicos segun estado
            if state == 0:
                ret = data_calma[idx_calma[sim, t]]
            else:
                ret = data_crisis[idx_crisis[sim, t]]
            
            cumret += np.dot(weights, ret)
            portfolio_values[sim, t+1] = np.exp(cumret)
    
    return portfolio_values

# Convertir a arrays numpy
data_calma_arr = data_calma.values
data_crisis_arr = data_crisis.values

print(f"Simulando {N_SIMULATIONS:,} trayectorias...")
portfolio_values = simulate_portfolio(
    N_SIMULATIONS, HORIZON_DAYS,
    hmm_model.transmat_, data_calma_arr, data_crisis_arr,
    initial_state=states[-1]
)
print("Completado")

In [ ]:
# ==============================================================================
# ALTERNATIVA: Simulacion con copulas (version parametrica)
# ==============================================================================
# Esta version ajusta copulas gaussianas a cada estado y genera muestras
# sinteticas. Es metodologicamente equivalente pero MUCHO mas lenta
# (tiempo de ejecucion ~10-100x mayor debido a copula.sample()).
# Se deja comentada como referencia.

# from copulas.multivariate import GaussianMultivariate

# def simulate_portfolio_copula(n_sim, horizon, transmat, copula_calma, copula_crisis, initial_state=0):
#     """
#     Simula trayectorias usando copulas ajustadas.
#     ADVERTENCIA: Muy lento para n_sim * horizon elevado
#     """
#     n_assets = 6
#     weights = np.ones(n_assets) / n_assets
#     portfolio_values = np.ones((n_sim, horizon + 1))
#
#     for sim in range(n_sim):
#         state = initial_state
#         cumret = 0.0
#         for t in range(horizon):
#             if np.random.random() < transmat[state, 1-state]:
#                 state = 1 - state
#
#             # Generar muestra desde la copula del estado activo
#             if state == 0:
#                 sample = copula_calma.sample(1).values.flatten()
#             else:
#                 sample = copula_crisis.sample(1).values.flatten()
#
#             cumret += np.dot(weights, sample)
#             portfolio_values[sim, t+1] = np.exp(cumret)
#
#     return portfolio_values

# Para usar esta version:
# copula_calma = GaussianMultivariate()
# copula_calma.fit(data_calma)
# copula_crisis = GaussianMultivariate()
# copula_crisis.fit(data_crisis)
# portfolio_values = simulate_portfolio_copula(...)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

ax1 = axes[0]
for i in range(min(300, N_SIMULATIONS)):
    color = COLOR_CRISIS if portfolio_values[i, -1] < 1 else COLOR_CALMA
    ax1.plot(portfolio_values[i, :], color=color, alpha=0.1, linewidth=0.5)
ax1.axhline(y=1, color='black', linestyle='--')
ax1.set_title('Muestra de Trayectorias Simuladas', fontweight='bold')
ax1.set_xlabel('Dias'); ax1.set_ylabel('Valor cartera')

ax2 = axes[1]
final_returns = (portfolio_values[:, -1] - 1) * 100
ax2.hist(final_returns, bins=80, color=COLOR_CALMA, alpha=0.7)
ax2.axvline(x=0, color='black', linestyle='--')
var99 = np.percentile(final_returns, 1)
ax2.axvline(x=var99, color=COLOR_CRISIS, linewidth=2, label=f'VaR 99%: {var99:.1f}%')
ax2.set_title('Distribucion de Retornos', fontweight='bold')
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
var_95 = np.percentile(final_returns, 5)
var_99 = np.percentile(final_returns, 1)
cvar_95 = final_returns[final_returns <= var_95].mean()
cvar_99 = final_returns[final_returns <= var_99].mean()

print("METRICAS DE RIESGO (Simulacion Base)")
print("="*50)
print(f"VaR 95%:  {var_95:.2f}%")
print(f"VaR 99%:  {var_99:.2f}%")
print(f"CVaR 95%: {cvar_95:.2f}%")
print(f"CVaR 99%: {cvar_99:.2f}%")
print(f"\nMedia: {final_returns.mean():.2f}% | Peor: {final_returns.min():.2f}%")

---
## 7. Escenarios de estres

In [ ]:
def stress_scenario(name, transmat_stressed, data, initial_state=1):
    """Simula escenario de estres."""
    n_assets = data.shape[1]
    weights = np.ones(n_assets) / n_assets
    idx = np.random.randint(0, len(data), size=(N_SIMULATIONS, HORIZON_DAYS))
    rand_trans = np.random.random((N_SIMULATIONS, HORIZON_DAYS))
    
    portfolio = np.ones((N_SIMULATIONS, HORIZON_DAYS + 1))
    for sim in range(N_SIMULATIONS):
        state = initial_state
        cumret = 0.0
        for t in range(HORIZON_DAYS):
            if rand_trans[sim, t] < transmat_stressed[state, 1-state]:
                state = 1 - state
            cumret += np.dot(weights, data[idx[sim, t]])
            portfolio[sim, t+1] = np.exp(cumret)
    
    final_ret = (portfolio[:, -1] - 1) * 100
    v99 = np.percentile(final_ret, 1)
    cv99 = final_ret[final_ret <= v99].mean()
    print(f"{name}: VaR99={v99:.1f}%, CVaR99={cv99:.1f}%, Peor={final_ret.min():.1f}%")
    return final_ret

In [ ]:
# Escenario 1: Estanflacion
transmat_estanf = np.array([[0.7, 0.3], [0.02, 0.98]])
ret_estanf = stress_scenario("Estanflacion 2022", transmat_estanf, data_crisis_arr)

# Escenario 2: Crisis de credito
transmat_credit = np.array([[0.5, 0.5], [0.01, 0.99]])
ret_credit = stress_scenario("Crisis Credito 2008", transmat_credit, data_crisis_arr)

# Escenario 3: Shock extremo (vol +50%)
data_extreme = data_crisis_arr * 1.5
transmat_extreme = np.array([[0.3, 0.7], [0.01, 0.99]])
ret_extreme = stress_scenario("Shock Extremo", transmat_extreme, data_extreme)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
scenarios = {'Base': final_returns, 'Estanflacion': ret_estanf, 'Crisis Credito': ret_credit, 'Shock Extremo': ret_extreme}
colors = [COLOR_CALMA, '#FFB347', '#FF6B6B', '#8B0000']

for i, (name, data) in enumerate(scenarios.items()):
    ax.hist(data, bins=80, alpha=0.5, color=colors[i], label=f'{name} (VaR99: {np.percentile(data, 1):.1f}%)')
ax.axvline(x=0, color='black', linestyle='--')
ax.set_xlabel('Retorno (%)')
ax.set_title('Comparacion de Escenarios', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
summary_scenarios = pd.DataFrame({
    'VaR 99%': [np.percentile(s, 1) for s in scenarios.values()],
    'CVaR 99%': [s[s <= np.percentile(s, 1)].mean() for s in scenarios.values()],
    'Peor Caso': [s.min() for s in scenarios.values()],
    'Media': [s.mean() for s in scenarios.values()]
}, index=scenarios.keys()).round(2)

print("RESUMEN ESCENARIOS")
display(summary_scenarios)

---
## 8. Conclusiones

In [ ]:
print("="*70)
print("CONCLUSIONES")
print("="*70)
print(f"\n1. REGIMENES: El HMM detecta dos estados claros.")
print(f"   Volatilidad crisis = {vol_crisis/vol_calma:.1f}x calma")
print(f"   Duracion crisis ~ {dur_crisis:.0f} dias")
print(f"\n2. ACTIVOS: El oro reduce correlacion en crisis (refugio).")
print(f"   High Yield se comporta como renta variable en crisis.")
print(f"\n3. RIESGO BASE: VaR 99% = {var_99:.2f}%, CVaR 99% = {cvar_99:.2f}%")
print(f"   Shock extremo: perdidas hasta {ret_extreme.min():.1f}%")
print(f"\n4. RECOMENDACIONES:")
print(f"   - Monitorizar cambio de regimen")
print(f"   - Coberturas de cola para escenarios extremos")
print(f"   - Reducir High Yield en anticipacion de crisis")
print(f"\nEjecutado: {datetime.now().strftime('%Y-%m-%d %H:%M')}")